In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Add ONE new insight explicitly (no scanning).
You provide: --title, --date, --category, --filename
- Dedup by (normalized title, normalized date).
- Append exactly one record with placeholders.
- Sort by date ASC and re-ID.
- Backup insights.json before write.
"""

import argparse, sys, json, unicodedata, re
from pathlib import Path
from datetime import datetime

# ---- Defaults (edit if your repo path differs)
BASE_DIR_DEFAULT = Path("/Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research")
INSIGHTS_JSON_SUB = Path("app") / "data" / "insights.json"
CHARTS_DIR_SUB = Path("public") / "charts"   # used to build /charts/<Category>/<filename> path string

CATEGORY_COLORS = {
    "Equities": "bg-blue-600",
    "Economics": "bg-emerald-600",
    "Technical": "bg-purple-600",
    "Commodities": "bg-amber-600",
    "Fixed Income": "bg-indigo-600",
    "Currencies": "bg-cyan-600",
    "Cross Assets": "bg-pink-600",
    "Education": "bg-orange-600",
}

# ---- unicode normalization helpers
DASH_MAP = {
    "\u2010": "-", "\u2011": "-", "\u2012": "-", "\u2013": "-", "\u2014": "-", "\u2212": "-",
}
QUOTE_MAP = { "\u2018": "'", "\u2019": "'", "\u201C": '"', "\u201D": '"' }
SPACE_RE = re.compile(r"\s+")

def nfkc(s: str) -> str:
    return unicodedata.normalize("NFKC", s or "")

def norm_text(s: str) -> str:
    s = nfkc(s)
    for k, v in {**DASH_MAP, **QUOTE_MAP}.items():
        s = s.replace(k, v)
    s = SPACE_RE.sub(" ", s).strip()
    return s

def normalize_date_any(date_str: str) -> str:
    """Accept YYYY-mm-dd or dd-mm-YYYY; return dd-mm-YYYY."""
    s = (date_str or "").strip()
    for fmt in ("%Y-%m-%d", "%d-%m-%Y"):
        try:
            return datetime.strptime(s, fmt).strftime("%d-%m-%Y")
        except ValueError:
            pass
    # try flexible split
    parts = [p for p in re.split(r"[^\d]", s) if p]
    if len(parts) == 3:
        # guess YYYY,MM,DD by 4-digit year
        year = next((p for p in parts if len(p) == 4), None)
        if year:
            rest = [p for p in parts if p != year]
            if len(rest) == 2:
                m, d = rest
                return datetime(int(year), int(m), int(d)).strftime("%d-%m-%Y")
    raise SystemExit(f"Unrecognized date: {date_str}")

def main():
    ap = argparse.ArgumentParser(description="Add one explicit insight safely.")
    ap.add_argument("--base", default=str(BASE_DIR_DEFAULT), help="Repo base dir")
    ap.add_argument("--insights-sub", default=str(INSIGHTS_JSON_SUB), help="Insights JSON subpath from base")
    ap.add_argument("--charts-sub", default=str(CHARTS_DIR_SUB), help="Charts dir subpath (to build /charts/... path)")

    # Required new-insight fields
    ap.add_argument("--title", required=True, help="Insight/Chart title (exact as you want displayed)")
    ap.add_argument("--date", required=True, help="Date of chart (YYYY-mm-dd or dd-mm-YYYY)")
    ap.add_argument("--category", required=True, choices=list(CATEGORY_COLORS.keys()), help="Category")
    ap.add_argument("--filename", required=True, help="Exact filename in that category folder (e.g. 'My Title_2025-10-20.svg')")

    # Optional
    ap.add_argument("--height", default=None, help="chartHeight (default auto: 700 if 'philly|ism|vvix|skew' in title else 350)")
    ap.add_argument("--apply", action="store_true", help="Write changes (default: dry-run)")

    # tolerate Jupyter's --f=
    args, unknown = ap.parse_known_args()
    if unknown:
        print(f"ℹ️  Ignoring unknown args: {unknown}")

    base = Path(args.base)
    insights_json = (base / Path(args.insights_sub)).resolve()
    charts_sub = Path(args.charts_sub)  # we only need it to build the /charts/<Category>/<filename> string

    if not insights_json.exists():
        raise SystemExit(f"insights.json not found: {insights_json}")

    # Load current JSON
    insights = json.loads(insights_json.read_text(encoding="utf-8"))
    if not isinstance(insights, list):
        raise SystemExit("insights.json is not a list.")

    # Normalize incoming fields
    title_disp = args.title.strip()
    title_key  = norm_text(title_disp)
    date_disp  = normalize_date_any(args.date)  # dd-mm-YYYY
    category   = args.category
    category_color = CATEGORY_COLORS[category]
    filename   = args.filename.strip()

    # Validate filename consistency with date in name (if it follows Title_YYYY-mm-dd.svg)
    # (Not required, but warns if mismatch)
    m = re.match(r"^(.+?)_(\d{4})[-_](\d{2})[-_](\d{2})\.(svg|png)$", filename, flags=re.IGNORECASE)
    if m:
        y, mm, dd = m.group(2), m.group(3), m.group(4)
        date_from_name = f"{dd}-{mm}-{y}"
        if date_from_name != date_disp:
            print(f"⚠️  Filename date {date_from_name} != provided date {date_disp} (using {date_disp}).")

    # Build chartPath (public URL path your app expects)
    chart_path = f"/charts/{category}/{filename}"

    # Check duplicate by (normalized title, normalized date)
    def key(ins):
        return (norm_text(ins.get("title","")), norm_text(ins.get("date","")))
    already = [ins for ins in insights if key(ins) == (title_key, date_disp)]
    if already:
        print("❌ An insight with the same (title,date) already exists. No changes made.")
        print(f"   Title: {title_disp}\n   Date:  {date_disp}")
        return

    # Decide height
    if args.height:
        chart_height = str(args.height)
    else:
        tl = title_key.lower()
        chart_height = "700" if any(w in tl for w in ["philly","ism","vvix","skew"]) else "350"

    # Create the new record
    new_entry = {
        "date": date_disp,
        "category": category,
        "categoryColor": category_color,
        "title": title_disp,
        "summary": f"**{title_disp}** — Add summary here.",
        "fullContent": f"**{title_disp}** — Add full content here. Edit insights.json to replace this placeholder.",
        "chartPath": chart_path,
        "chartHeight": chart_height,
    }

    # Merge, sort by date ASC, re-ID
    merged = insights + [new_entry]
    def sort_key(ins):
        try:
            return datetime.strptime(ins.get("date","01-01-1900"), "%d-%m-%Y")
        except Exception:
            return datetime(1900,1,1)
    merged.sort(key=sort_key)
    for i, ins in enumerate(merged, 1):
        ins["id"] = i

    # Preview
    print("\n✅ Will add exactly ONE insight:")
    print(f"   Title: {title_disp}")
    print(f"   Date:  {date_disp}")
    print(f"   Cat:   {category} ({category_color})")
    print(f"   Path:  {chart_path}")
    print(f"   Height:{chart_height}")
    print(f"📊 New total after insert: {len(merged)}")

    if args.apply:
        # Backup
        ts = datetime.now().strftime("%Y%m%d-%H%M%S")
        backup = insights_json.with_suffix(f".json.bak-{ts}")
        backup.write_text(insights_json.read_text(encoding="utf-8"), encoding="utf-8")
        print(f"\n🛟 Backup saved: {backup}")
        # Write
        insights_json.write_text(json.dumps(merged, indent=2, ensure_ascii=False), encoding="utf-8")
        print(f"💾 insights.json updated: {insights_json}")
    else:
        print("\nℹ️ Dry-run only. Use --apply to write changes.")

if __name__ == "__main__":
    main()
